In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score,accuracy_score,roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans
import random
from sklearn.svm import OneClassSVM

In [2]:
from imblearn.datasets import fetch_datasets
fetch_dataset=fetch_datasets()
#fetch_dataset=readfile("fetch_data.pkl")
data_name=[]
for word in fetch_dataset:
    print(word)
    data_name.append(word)

ecoli
optical_digits
satimage
pen_digits
abalone
sick_euthyroid
spectrometer
car_eval_34
isolet
us_crime
yeast_ml8
scene
libras_move
thyroid_sick
coil_2000
arrhythmia
solar_flare_m0
oil
car_eval_4
wine_quality
letter_img
yeast_me2
webpage
ozone_level
mammography
protein_homo
abalone_19


In [4]:
%%time
e=0
word="protein_homo"
X=fetch_dataset[word]["data"]
Y=fetch_dataset[word]["target"]
print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
X[pd.isnull(X)]=0
random.seed(0)
d=len(X[0])
split=0.4
#random_value=1
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
positive=X_train[y_train==1]
negative=X_train[y_train==-1]

protein_homo : 1296 : 144455 : 74
CPU times: user 152 ms, sys: 39.4 ms, total: 192 ms
Wall time: 191 ms


In [ ]:
makefile(negative,"protein_train.pkl")

# OCSVM

In [9]:
%%time
OCSVM=OneClassSVM()
OCSVM.fit(negative)

CPU times: user 7min 32s, sys: 55.5 ms, total: 7min 32s
Wall time: 7min 32s


OneClassSVM()

In [ ]:
makefile(OCSVM,"protein_OCSVM.pkl")

In [10]:
%%time
score=OCSVM.score_samples(X_test)
print(roc_auc_score(y_test,-score))

0.8652633735443559
CPU times: user 2min 18s, sys: 4 ms, total: 2min 18s
Wall time: 2min 18s


# ODSVM all

In [12]:
%%time
models=[]
#avg=negative.mean(axis=0)

#OCSVM=OneClassSVM()
#OCSVM.fit(avg.reshape(1,-1))
#models.append(OCSVM)
#first=OCSVM.score_samples(negative)
#index=first.argsort().argsort()

#OCSVM=OneClassSVM()

#OCSVM.fit(negative[index].reshape(1,-1))
#models.append(OCSVM)

#scores=OCSVM.score_samples(negative)

#support=negative[index<200]
for data in negative:
    OCSVM=OneClassSVM()
    OCSVM.fit(data.reshape(1,-1))
    models.append(OCSVM)

CPU times: user 27.1 s, sys: 83.9 ms, total: 27.2 s
Wall time: 27.2 s


In [ ]:
makefile(models,"protein_all.pkl")

In [13]:
%%time
#summation
ens=np.zeros(len(X_test))
#num_base=[]
for model in models:
    ens+=model.score_samples(X_test)
    #num_base.append(model.score_samples(X_test))
#ens=np.array(num_base).sum(axis=0)
#print(len(models))
print(roc_auc_score(y_test,-ens))

0.8532115930745285
CPU times: user 9min 43s, sys: 11.8 ms, total: 9min 43s
Wall time: 9min 43s


# ODSVM N

In [8]:
%%time
models=[]
avg=negative.mean(axis=0)

OCSVM=OneClassSVM()
OCSVM.fit(avg.reshape(1,-1))
#models.append(OCSVM)
first=OCSVM.score_samples(negative)
index=first.argsort().argsort()

N=200

support=negative[index<N]
for data in support:
    OCSVM=OneClassSVM()
    OCSVM.fit(data.reshape(1,-1))
    models.append(OCSVM)

CPU times: user 85.3 ms, sys: 0 ns, total: 85.3 ms
Wall time: 84.3 ms


In [ ]:
makefile(models,"protein_n.pkl")

In [6]:
%%time
#summation
ens=np.zeros(len(X_test))
num_base=[]
for model in models:
    ens+=model.score_samples(X_test)
    #num_base.append(model.score_samples(X_test))
#ens=np.array(num_base).sum(axis=0)
#print(len(models))
print(roc_auc_score(y_test,-ens))


0.681066334121731
CPU times: user 1.36 s, sys: 0 ns, total: 1.36 s
Wall time: 1.36 s


In [16]:
%%time
#maximum
#ens=np.zeros(len(X_test))
num_base=[]
for model in models:
    num_base.append(model.score_samples(X_test))
ens=np.array(num_base).sum(axis=0)
#print(len(models))
print(roc_auc_score(y_test,-ens))



0.6455120632733425
CPU times: user 6.71 s, sys: 36 ms, total: 6.75 s
Wall time: 6.75 s
